# TP : Sentiment analysis on IMDB movie reviews

## Objectives

1. Implement a simple way to represent text data - Bag of words
2. Implement a basic statistical learning model - Bayesian Naive
3. Use these representations and this model for a sentiment analysis task.
4. Implement different ways of obtaining dense representations of the same data
5. Use a logistic regression model to train a classifier on these new representations.

## Necessary dependancies

We will need the following packages:
- The Machine Learning API Scikit-learn : http://scikit-learn.org/stable/install.html
- The Natural Language Toolkit : http://www.nltk.org/install.html

Both are available with Anaconda: https://anaconda.org/anaconda/nltk and https://anaconda.org/anaconda/scikit-learn

In [ ]:
import os.path as op
import re 
import numpy as np
import matplotlib.pyplot as plt

## Loading data

We retrieve the textual data in the variable *texts*.

The labels are retrieved in the variable $y$ - it contains *len(texts)* of them: $0$ indicates that the corresponding review is negative while $1$ indicates that it is positive.

In [ ]:
from glob import glob
# We get the files from the path: ./data/imdb1/neg for negative reviews, and ./data/imdb1/pos for positive reviews
filenames_neg = sorted(glob(op.join('.', 'data', 'imdb1', 'neg', '*.txt'))) # recupérer les fichiers
filenames_pos = sorted(glob(op.join('.', 'data', 'imdb1', 'pos', '*.txt')))

# Each files contains a review that consists in one line of text: we put this string in two lists, that we concatenate
texts_neg = [open(f, encoding="utf8").read() for f in filenames_neg] # recupérer le fichier d'un coup, on peut utiliser readlines()
texts_pos = [open(f, encoding="utf8").read() for f in filenames_pos]
texts = texts_neg + texts_pos # concatener lists 

# The first half of the elements of the list are string of negative reviews, and the second half positive ones
# We create the labels, as an array of [1,len(texts)], filled with 1, and change the first half to 0
y = np.ones(len(texts), dtype=int) #première motié des zeros 
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

In [ ]:
# This number of documents may be high for most computers: we can select a fraction of them (here, one in k)
# Use an even number to keep the same number of positive and negative reviews
k = 100
texts_reduced = texts[0::k]
y_reduced = y [0::k]

print('Number of documents:', len(texts_reduced))

In [ ]:
from sklearn.model_selection import train_test_split
train_texts_splt, val_texts, train_labels_splt, val_labels = train_test_split(texts_reduced, y_reduced, test_size=.2)
# 20% de données 

In [ ]:
# P(c) frequanece dans classe 
# Hypothesebaysienne: probabilité de most parce que most sont indépendants entre eux
# nombre de parameres = v * c (nombre vocabulaore ) * nombre classe s

# Naive Bayesian 

## Main idea

A movie review is in fact a list of words $s = (w_1, ..., w_N)$, and we try to find the associated class $c$ - which in our case may be $c = 0$ or $c = 1$. The objective is thus to find for each review $s$ the class $\hat{c}$ maximizing the conditional probability **$P(c|s)$** : 

$$\hat{c} = \underset{c}{\mathrm{argmax}}\, P(c|s) = \underset{c}{\mathrm{argmax}}\,\frac{P(s|c)P(c)}{P(s)}$$

**Hypothesis : P(s) is constant for each class** :

$$\hat{c} = \underset{c}{\mathrm{argmax}}\,\frac{P(s|c)P(c)}{P(s)} = \underset{c}{\mathrm{argmax}}\,P(s|c)P(c)$$

**Naive hypothesis : the variables (words) of a review are independant between themselves** : 

$$P(s|c) = P(w_1, ..., w_N|c)=\Pi_{i=1..N}P(w_i|c)$$

We will therefore be able to use the reviews at our disposal to **estimate the probabilities $P(w|c)$ for each word $w$ given the two classes $c$**. These reviews will allow us to learn how to evaluate the "compatibility" between words and classes.

## General view

### Training: Estimating the probabilities

For each word $w$ in the vocabulary $V$, $P(w|c)$ is the number of occurrences of $w$ in all reviews of class $c$, divided by the total number of occurrences in $c$. If we note $T(w,c)$ this number of occurrences, we get:

$$P(w|c) = \text{Frequency of }w\text{ in }c = \frac{T(w,c)}{\sum_{w' \in V} T(w',c)}$$

### Test: Calculating scores

To facilitate the calculations and to avoid *underflow* and approximation errors, we use the log-sum trick, and we pass the equation into log-probabilities : 

$$ \hat{c} = \underset{c}{\mathrm{argmax}} P(c|s) = \underset{c}{\mathrm{argmax}} \left[ \mathrm{log}(P(c)) + \sum_{i=1..N}log(P(w_i|c)) \right] $$

### Laplace smoothing

A word that does not appear in a document has a probability of zero: this will cause issues with the logarithm. So we keep a very small part of the probability mass that we redistribute with the *Laplace smoothing*: 

$$P(w|c) = \frac{T(w,c) + 1}{\sum_{w' \in V} (T(w',c) + 1)}$$

There are other smoothing methods, generally suitable for other, more complex applications. 

In [ ]:
# sum est sur mot sachant neg , le problème est quand la prob est zero
# tableau 2 * vocabumaire, faut vectorizer 
# calculer prob: vecteur phrase * log prob modèle 
# prob de phrase sachant : produit de matrices 

## Adapted representation of documents

Our statistical model, like most models applied to textual data, uses counts of word occurrences in a document. Thus, a very convenient way to represent a document is to use a Bag-of-Words (BoW) vector, containing the counts of each word (regardless of their order of occurrence) in the document. 

If we consider the set of all the words appearing in our $T$ training documents, which we note $V$ (Vocabulary), we can create **an index**, which is a bijection associating to each $w$ word an integer, which will be its position in $V$. 

Thus, for a document extracted from a set of documents containing $|V|$ different words, a BoW representation will be a vector of size $|V|$, whose value at the index of a word $w$ will be its number of occurrences in the document. 

We can use the **CountVectorizer** class from scikit-learn to better understand:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer # matrix de compte de données 

from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
corpus = ['I walked down down the boulevard',
          'I walked down the avenue',
          'I ran down the boulevard',
          'I walk down the city',
          'I walk down the the avenue']
vectorizer = CountVectorizer()

Bow = vectorizer.fit_transform(corpus) # tockenizer les phrase , bag of word

print(vectorizer.get_feature_names()) # obtenir les features, mots dans le vocabulaiore , il retire les mots cours ("I" disparait)
Bow.toarray()

We display the list containing the words ordered according to their index (Note that words of 2 characters or less are not counted).

## Detail: training

The idea is to extract the number of occurrences $T(w,c)$ for each word $w$ and each class $c$, which will make it possible to calculate the matrix of conditional probabilities $\pmb{P}$ such that: $$\pmb{P}_{w,c} = P(w|c)$$

Note that the number of occurrences $T(w,c)$ can be easily obtained from the BoW representations of all documents !

### Procedure:

- Extract the vocabulary $V$ and counts $T(w,c)$ for each of the words $w$ and classes $c$, from a set of documents.
- Calculate the a priori probabilities of the classes $P(c) = \frac{\sum_{w \in V} T(w,c)}{\sum_{c \in C} \sum_{w \in V} T(w,c)}$
- Calculate the conditional **smoothed** probabilities $P(w|c) = \frac{T(w,c) + 1}{\sum_{w' \in V} (T(w',c) + 1)}$.

## Detail: test

We now know the conditional probabilities given by the $\pmb{P}$ matrix. 
Now we must obtain $P(s|c)$ for the current document. This quantity is obtained using a simple calculation involving the BoW representation of the document and $\pmb{P}$.

### Procedure:

- For each of the classes $c$,
    - $Score(c) = \log P(c)$
    - For each word $w$ in the document to be tested:
        - $Score(c) += \log P(w|c)$
- Return $argmax_{c \in C} Score(c)$ 

In [ ]:
# preprocessing,: comment créer un bag of word

## Preprocessing the text: get the BoW representations ##

The first thing to do is to turn the review from a string into a list of words. The simplest method is to divide the string according to spaces with the command:
``text.split()``

But we must also be careful to remove special characters that may not have been cleaned up (such as HTML tags if the data was obtained from web pages). Since we're going to count words, we'll have to build a list of tokens appearing in our data. In our case, we'd like to reduce this list and make it uniform (ignore capitalization, punctuation, and the shortest words). 
We will therefore use a function adapted to our needs - but this is a job that we generally don't need to do ourselves, since there are many tools already adapted to most situations. 
For text cleansing, there are many scripts, based on different tools (regular expressions, for example) that allow you to prepare data. The division of the text into words and the management of punctuation is handled in a step called *tokenization*; if needed, a python package like NLTK contains many different *tokenizers*.

In [ ]:
# Careful, NLTK was the first well-documented NLP package, but it might be outdated for some uses.
# Check the documentation !
from nltk.tokenize import word_tokenize # pour obtenir tagging rapide, ploen de tokinzer , faut telechatder de donneés pour l'uilizer 

corpus_raw = "I walked down down the boulevard. I walked down the avenue. I ran down the boulevard. I walk down the city. I walk down the the avenue."
print(word_tokenize(corpus_raw))

The next function takes as input a list of documents (each in the form of a string) and returns, as in the example using ``CountVectorizer``:
- A vocabulary that associates, to each word encountered, an index
- A matrix, with rows representing documents and columns representing words indexed by the vocabulary. In position $(i,j)$, one should have the number of occurrences of the word $j$ in the document $i$.

The vocabulary, which was in the form of a *list* in the previous example, can be returned in the form of a *dictionary* whose keys are the words and values are the indices. Since the vocabulary lists the words in the corpus without worrying about their number of occurrences, it can be built up using a set (in python).

In [ ]:
def count_words(texts, voc = None):
    """Vectorize text : return count of each word in the text snippets

    Parameters
    ----------
    texts : list of str
        The texts
    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
    """
    n_samples = len(texts)
    if voc == None:
        words = set() # set de vobaulaire
        for text in texts:
            words = words.union(set(word_tokenize(text))) # list of all words, unioni de l'ensemble et avec le text 
        n_features = len(words) # number of different words
        vocabulary = dict(zip(words, range(n_features))) # vocab[wd] = index ; indexisation, associer à chaque mot un nombre (index), 
                                                            # correspances entre mot et dimension
    else: # créer la matrix , vocabulaire associé aux données d'éntreé 
        vocabulary = voc
        n_features = len(voc)
    counts = np.zeros((n_samples, n_features))
    for k, text in enumerate(texts): # enumeration a k for a text[k]
        for w in word_tokenize(text):
            if w in vocabulary:
                counts[k][vocabulary[w]] += 1.
    return vocabulary, counts

In [ ]:
Voc, X = count_words(corpus)
print(Voc)
print(X)

## Naive Bayesian 


```python
def fit(self, X, y)
``` 
**Training**: will learn a statistical model based on the representations $X$ corresponding to the labels $y$.
Here, $X$ contains representations obtained as the output of ```count_words```. You can complete the function using the procedure detailed above. 

Note: the smoothing is not necessarily done with a $1$ - it can be done with a positive value $\alpha$, which we can implement as an argument of the class "NB".

```python
def predict(self, X)
```
**Testing**: will return the labels predicted by the model for other representations $X$.

In [ ]:
class NB(BaseEstimator, ClassifierMixin):
    # Les arguments de classe permettent l'héritage de classes de sklearn
    def __init__(self, alpha=1.0 #hyperparametre):
        # alpha est un paramètre pour le lissage: il correspond à la valeur ligne 10 de l'algorithme d'entraînement
        # Dans l'algorithme d'entraînement, et comme valeur par défaut, on utilise alpha = 1
        self.alpha = alpha

    def fit(self, X, y): # calculer frequence relatrive 
        n_samples, n_features = X.shape

        classes = np.unique(y) # all labels
        n_classes = len(classes) # number of labels

        prior = np.zeros(n_classes) # probabilité de classes

        tct = np.zeros((n_classes, n_features)) # tableau qui contient probabilité contitionnelle 
        for k, c in enumerate(classes):
            prior[k] = np.sum(y == c) / float(n_samples) # calcul probabilté de la classe
            tct[k, :] = np.sum(X[y == c, :], axis=0) # number(words|classe)

        alpha = self.alpha  # laplace smoothing / lissage de laplace
        cond_prob = (tct + alpha) / np.sum(tct + alpha, axis=1)[:, None] # version normalizé 

        self.prior_ = prior
        self.log_cond_prob_ = np.log(cond_prob)
        return self

    def predict(self, X):
        n_classes = len(self.prior_)
        scores = np.dot(X, self.log_cond_prob_.T) # produit scalaire entre bag of world et matricx conditionnelle 
        scores += np.log(self.prior_)[None, :]
        return np.array([0, 1])[np.argmax(scores, axis=1)] # armax pour faire la prediction

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

## Experimentation

We use half the data for training, and the other half for evaluation.

In [ ]:
voc, train_bow = count_words(train_texts_splt) # faut faire un peu de nottage, enlever mayuscules 

In [ ]:
_, val_bow = count_words(val_texts, voc)

In [ ]:
clf_nb = NB()
clf_nb.fit(train_bow, train_labels_splt)
val_pred = clf_nb.predict(val_bow)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
print(classification_report(val_labels, val_pred))
cm = confusion_matrix(val_labels , val_pred, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot()
plt.show()
# précision taux zero , vertical
# recal c'est horizontal 
# F1 ration, ponderation des deux
# macro average quand il y a une clsse plus requentes que les autres  

## Evaluating performances: 

**What are the strengths and weaknesses of this system? How can they be remedied?**

## To go further: 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

## Scikit-learn

### Improving representations

We use the function 
```python
CountVectorizer
``` 
of scikit-learn. It will allow us to easily improve our BoW representations.

#### Tf-idf:

This is the product of the frequency of the term (TF) and its inverse frequency in documents (IDF).
This method is usually used to measure the importance of a term $i$ in a document $j$ relative to the rest of the corpus, from a matrix of occurrences $ words \times documents$. Thus, for a matrix $\mathbf{T}$ of $|V|$ terms and $D$ documents:
$$\text{TF}(T, w, d) = \frac{T_{w,d}}{\sum_{w'=1}^{|V|} T_{w',d}} $$

$$\text{IDF}(T,w) = \log \left(\frac{D}{|\{d: T_{w,d} > 0\}|} \right)$$

$$\text{TF-IDF}(T, w, d) = \text{TF}(T, w, d) \cdot \text{IDF}(T, w)$$

It can be adapted to our case by considering that the context of the second word is the document. However, TF-IDF is generally better suited to low-density matrices, since it will penalize terms that appear in a large part of the documents. 
    
#### Do not take into account words that are too frequent:

We can use the argument
```python
max_df=1.0
```
to change the amount of words taken into account. 

#### Try different granularities:

Rather than just counting words, we can count sequences of words - limited in size, of course. 
We call a sequence of $n$ words a $n$-gram: let's try using 2 and 3-grams (bi- and trigrams).
We can also try to use character sequences instead of word sequences.

We will be interested in the options 
```python
analyze='word'
```
and 
```python
ngram_range=(1, 2)
```
which we'll change to alter the granularity. 

In [ ]:
## We can define a pipeline, with which we can experiment.

pipeline_tf_idf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', max_df=0.7, stop_words=None)),
    ('tfidf', TfidfTransformer()), # Améliorer la performance
    ('clf', MultinomialNB()),
])
pipeline_tf_idf.fit(train_texts_splt, train_labels_splt)
val_pred = pipeline_tf_idf.predict(val_texts)
print(classification_report(val_labels, val_pred))

### Natural Language Toolkit (NLTK)

### Stemming 

Allows to go back to the root of a word: you can group different words around the same root, which facilitates generalization. Use:
```python
from nltk import SnowballStemmer
```

In [ ]:
from nltk import SnowballStemmer # ça coupe les mots, résuit le vocabulaire 
stemmer = SnowballStemmer("english")

#### Example:

In [ ]:
words = ['singers', 'cat', 'generalization', 'philosophy', 'psychology', 'philosopher']
for word in words:
    print('word : %s ; stemmed : %s' %(word, stemmer.stem(word)))#.decode('utf-8'))))

#### Application:

In [ ]:
def stem(X): 
    X_stem = []
    for text in X:
        wordlist = []
        for word in text.split(" "):
            stemmword = stemmer.stem(word)#.decode('utf-8'))
            wordlist.append(stemmword)
        X_stem.append(' '.join(wordlist))
    return X_stem

In [ ]:
texts_stemmed = stem(train_texts_splt)
val_texts_stemmed = stem(val_texts)
voc, train_bow = count_words(texts_stemmed)
_, val_bow = count_words(val_texts_stemmed, voc)

In [ ]:
print(train_texts_splt[0])
print(texts_stemmed[0])

In [ ]:
clf_nb = NB()
clf_nb.fit(train_bow, train_labels_splt)
val_pred = clf_nb.predict(val_bow)

print(classification_report(val_labels, val_pred))
cm = confusion_matrix(val_labels , val_pred, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot()
plt.show()

### Part of speech tags

To generalize, we can also use the Part of Speech (POS) of the words, which will allow us to filter out information that is potentially not useful to the model. We will retrieve the POS of the words using the functions:
```python
from nltk import pos_tag, word_tokenize
```

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize

#### Example:

In [ ]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

pos_tag(word_tokenize(('I am Sam')))

Details of POS tags meanings: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk

#### Application:

Only keeps nouns, adverbs, verbs and adjectives for our model. 

In [ ]:
def pos_tag_filter(X, good_tags=['NN', 'VB', 'ADJ', 'RB']):
    X_pos = []
    for text in X:
        tags = nltk.pos_tag(nltk.word_tokenize(text))
        X_pos.append(' '.join([w for w, t in tags if t in good_tags]))
    return X_pos

In [ ]:
texts_POS = pos_tag_filter(train_texts_splt)
val_texts_POS = pos_tag_filter(val_texts)
voc, train_bow = count_words(texts_stemmed)
_, val_bow = count_words(val_texts_stemmed, voc)

clf_nb = NB()
clf_nb.fit(train_bow, train_labels_splt)
val_pred = clf_nb.predict(val_bow)

print(classification_report(val_labels, val_pred))
cm = confusion_matrix(val_labels , val_pred, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot()
plt.show()